In [1]:
from os import listdir; from IPython.display import display, HTML; from os.path import isfile, join
import pandas as pd, numpy as np, seaborn as sns, json, csv, os, ast, xml.etree.ElementTree
%matplotlib inline
import json
from matplotlib.backends.backend_pdf import PdfPages
sns.set_style('whitegrid')

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

from matplotlib import rcParams
rcParams['axes.titlepad'] = 20
from collections import Counter

In [2]:
df = pd.read_csv('../../Task/BigTask/Parsed-results_S6.csv')

gold_dict = ['HIGH', 'LOW']

In [3]:
# Calcolo Frequenze Relative
def compute_relatives(table):
    tot = table['worker_id'].sum()
    table['worker_id'] = table.apply(lambda row: round(row.worker_id / tot, 3), axis = 1)
    return table

# Funzione per Estrapolare il Dominio
def find_domain(url):
    st = url.split("/")
    return st[2]

my_df = df
my_df = my_df[my_df["doc_id"].isin(gold_dict) == False]
# URL 80 è FAKE
#my_df['url'][80] = "http://www.fakeURL.com"
#my_df['url'][373] = "http://www.fakeURL.com"
# Prendo i domini
my_df['url'] = my_df.apply(lambda row: find_domain(row.url), axis = 1)
#display(my_df)

poli_df = my_df[(my_df.url == "www.politifact.com")]
display(poli_df.head())

rest_df = my_df.merge(poli_df, indicator = True, how='left').loc[lambda x : x['_merge']!='both']
display(rest_df.head())

#poli_df.to_csv("../../Task/BigTask/Poli_S6.csv", index=False, header=True)
#rest_df.to_csv("../../Task/BigTask/Not_Poli_S6.csv", index=False, header=True)

c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,unit_id,doc_id,rel,worker_id,gender,age,school,income,party,party_ln,ideo,elections,teaparty,position_in_task,S6_rel,justification,url,off_page_count,ip_address,time
3,unit_0,DEM_HALFTRUE_doc3,HALFTRUE,AFBS8D6KBKKKY,2,3,4,3,2,2,4,1,3,4,4,"It was found by IFC that yes, water rates did ...",www.politifact.com,6,65.35.157.98,152.994
6,unit_0,REP_MOSTLYTRUE_doc5,MOSTLYTRUE,AFBS8D6KBKKKY,2,3,4,3,2,2,4,1,3,7,5,His turnaround was a pretty big one and well k...,www.politifact.com,2,65.35.157.98,117.847
7,unit_0,REP_BARELYTRUE_doc4,BARELYTRUE,AFBS8D6KBKKKY,2,3,4,3,2,2,4,1,3,8,4,The Secure Fence Act was a thing and yes it co...,www.politifact.com,1,65.35.157.98,161.113
16,unit_2,DEM_HALFTRUE_doc3,HALFTRUE,A2E9KS4RK6T8CW,1,4,4,7,4,2,3,1,3,1,6,A very true statement of Gwen Moore regarding ...,www.politifact.com,0,107.198.59.190,96.154
19,unit_2,DEM_FALSE_doc1,FALSE,A2E9KS4RK6T8CW,1,4,4,7,4,2,3,1,3,4,6,It is a true statement from Nacy Pelosi,www.politifact.com,1,107.198.59.190,55.187


,unit_id,doc_id,rel,worker_id,gender,age,school,income,party,party_ln,...,elections,teaparty,position_in_task,S6_rel,justification,url,off_page_count,ip_address,time,_merge
0,unit_0,REP_LIE_doc6,LIE,AFBS8D6KBKKKY,2,3,4,3,2,2,...,1,3,1,1,"After implementation of the law, 98% of Massac...",en.wikipedia.org,0,65.35.157.98,105.615,left_only
1,unit_0,DEM_FALSE_doc1,FALSE,AFBS8D6KBKKKY,2,3,4,3,2,2,...,1,3,2,1,She has literally cited him in the past.,twitchy.com,3,65.35.157.98,133.090,left_only
3,unit_0,DEM_TRUE_doc2,TRUE,AFBS8D6KBKKKY,2,3,4,3,2,2,...,1,3,5,2,Ryan sounds like he really does regret ever su...,www.vanityfair.com,3,65.35.157.98,122.103,left_only
6,unit_1,REP_TRUE_doc8,TRUE,A1HU0I8CI303XM,2,3,5,7,3,1,...,1,1,1,5,ITS MOSTLY CRT,en.wikipedia.org,0,192.3.161.207,414.396,left_only
7,unit_1,REP_FALSE_doc2,FALSE,A1HU0I8CI303XM,2,3,5,7,3,1,...,1,1,3,6,ITS VERY INTERESTING,en.wikipedia.org,5,192.3.161.207,139.195,left_only


In [26]:
def check_same(str1, str2):
    if(str1 == str2):
        return True
    else:
        return False

poli_df['S6_rel'] = poli_df['S6_rel'].map({1: 'LIE', 2: 'FALSE', 3: 'BARELYTRUE', 4: 'HALFTRUE', 5: 'MOSTLYTRUE', 6: 'TRUE'})
rest_df['S6_rel'] = rest_df['S6_rel'].map({1: 'LIE', 2: 'FALSE', 3: 'BARELYTRUE', 4: 'HALFTRUE', 5: 'MOSTLYTRUE', 6: 'TRUE'})

poli_df['same'] = poli_df.apply(lambda row: check_same(row.rel, row.S6_rel), axis = 1)
rest_df['same'] = rest_df.apply(lambda row: check_same(row.rel, row.S6_rel), axis = 1)

same_count_poli = poli_df.groupby(['same']).count().reset_index()
display(same_count_poli)
same_count_other = rest_df.groupby(['same']).count().reset_index()
display(same_count_other)

c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,same,unit_id,doc_id,rel,worker_id,gender,age,school,income,party,...,ideo,elections,teaparty,position_in_task,S6_rel,justification,url,off_page_count,ip_address,time
0,False,411,411,411,411,411,411,411,411,411,...,411,411,411,411,411,411,411,411,411,411
1,True,471,471,471,471,471,471,471,471,471,...,471,471,471,471,471,471,471,471,471,471


,same,unit_id,doc_id,rel,worker_id,gender,age,school,income,party,...,elections,teaparty,position_in_task,S6_rel,justification,url,off_page_count,ip_address,time,_merge
0,False,258,258,258,258,258,258,258,258,258,...,258,258,258,258,258,258,258,258,258,258
1,True,60,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60


In [4]:
# Calcolo Frequenze Relative
def compute_relatives(table):
    tot = table['worker_id'].sum()
    table['worker_id'] = table.apply(lambda row: round(row.worker_id / tot, 3), axis = 1)
    return table

# Funzione per Estrapolare il Dominio
def find_domain(url):
    st = url.split("/")
    return st[2]
df = pd.read_csv('../../Task/BigTask/Parsed-results_S100.csv')

gold_dict = ['HIGH', 'LOW']

my_df = df
# URL 80 è FAKE
my_df['url'][80] = "http://www.fakeURL.com"
my_df['url'][373] = "http://www.fakeURL.com"
# Prendo i domini
my_df['url'] = my_df.apply(lambda row: find_domain(row.url), axis = 1)
#display(my_df)
poli_df = my_df[(my_df.url == "www.politifact.com")]
display(poli_df.head())

rest_df = my_df.merge(poli_df, indicator = True, how='left').loc[lambda x : x['_merge']!='both']
display(rest_df.head())

#poli_df.to_csv("../../Task/BigTask/Poli_S100.csv", index=False, header=True)
#rest_df.to_csv("../../Task/BigTask/Not_Poli_S100.csv", index=False, header=True)

c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,unit_id,doc_id,rel,worker_id,gender,age,school,income,party,party_ln,ideo,elections,teaparty,position_in_task,S100_rel,justification,url,off_page_count,ip_address,time
3,unit_0,DEM_HALFTRUE_doc3,HALFTRUE,A28KV1QXL1AV4X,2,5,3,6,3,1,2,1,1,4,2.0,Looks like it went up 170%.according to Politi...,www.politifact.com,1,74.97.48.154,147.263
6,unit_0,REP_MOSTLYTRUE_doc5,MOSTLYTRUE,A28KV1QXL1AV4X,2,5,3,6,3,1,2,1,1,7,93.0,"He said that in a January 28, 2008, speech.",www.politifact.com,3,74.97.48.154,125.867
8,unit_1,REP_TRUE_doc8,TRUE,A10LHWALI4BZPC,1,3,6,9,1,1,2,1,1,1,100.0,\r\nThe average annual pension benefit for new...,www.politifact.com,0,73.129.121.170,25.925
10,unit_1,REP_FALSE_doc2,FALSE,A10LHWALI4BZPC,1,3,6,9,1,1,2,1,1,3,10.0,"Richardson, in criticizing Kitzhaber’s years a...",www.politifact.com,0,73.129.121.170,21.817
11,unit_1,DEM_BARELYTRUE_doc9,BARELYTRUE,A10LHWALI4BZPC,1,3,6,9,1,1,2,1,1,4,19.0,"McAuliffe did not acknowledge these points, al...",www.politifact.com,0,73.129.121.170,25.283


,unit_id,doc_id,rel,worker_id,gender,age,school,income,party,party_ln,...,elections,teaparty,position_in_task,S100_rel,justification,url,off_page_count,ip_address,time,_merge
0,unit_0,REP_LIE_doc6,LIE,A28KV1QXL1AV4X,2,5,3,6,3,1,...,1,1,1,3.0,"Rick Perry was governor of Texas, not Massachu...",en.wikipedia.org,0,74.97.48.154,59.715,left_only
1,unit_0,DEM_FALSE_doc1,FALSE,A28KV1QXL1AV4X,2,5,3,6,3,1,...,1,1,2,6.0,Nancy Pelosi is House Majority leader and Spea...,www.facebook.com,1,74.97.48.154,85.715,left_only
2,unit_0,LOW,LOW,A28KV1QXL1AV4X,2,5,3,6,3,1,...,1,1,3,1.0,"Lincoln lived in the 19th century, at which ti...",www.anb.org,1,74.97.48.154,86.520,left_only
4,unit_0,DEM_TRUE_doc2,TRUE,A28KV1QXL1AV4X,2,5,3,6,3,1,...,1,1,5,40.0,I cannot find anything that supports this as f...,www.vanityfair.com,3,74.97.48.154,291.685,left_only
5,unit_0,HIGH,HIGH,A28KV1QXL1AV4X,2,5,3,6,3,1,...,1,1,6,96.0,See speech in New Hampshire.,www.youtube.com,6,74.97.48.154,158.746,left_only


In [4]:
df = pd.read_csv('../../Task/BigTask/Poli_S6.csv')

m = df.groupby(['doc_id']).count()
m = m['unit_id']
print(m.mean())

7.35


In [5]:
df = pd.read_csv('../../Task/BigTask/Poli_S100.csv')

m = df.groupby(['doc_id']).count()
m = m['unit_id']
print(m.mean())

7.188524590163935


In [9]:
df = pd.read_csv('../../Task/BigTask/Not_Poli_S6.csv')

m = df.groupby(['doc_id']).count()
m = m['unit_id']
display(df.head())
print(m.mean())

,unit_id,doc_id,rel,worker_id,gender,age,school,income,party,party_ln,...,elections,teaparty,position_in_task,S6_rel,justification,url,off_page_count,ip_address,time,_merge
0,unit_0,REP_LIE_doc6,LIE,AFBS8D6KBKKKY,2,3,4,3,2,2,...,1,3,1,1,"After implementation of the law, 98% of Massac...",en.wikipedia.org,0,65.35.157.98,105.615,left_only
1,unit_0,DEM_FALSE_doc1,FALSE,AFBS8D6KBKKKY,2,3,4,3,2,2,...,1,3,2,1,She has literally cited him in the past.,twitchy.com,3,65.35.157.98,133.090,left_only
2,unit_0,DEM_TRUE_doc2,TRUE,AFBS8D6KBKKKY,2,3,4,3,2,2,...,1,3,5,2,Ryan sounds like he really does regret ever su...,www.vanityfair.com,3,65.35.157.98,122.103,left_only
3,unit_1,REP_TRUE_doc8,TRUE,A1HU0I8CI303XM,2,3,5,7,3,1,...,1,1,1,5,ITS MOSTLY CRT,en.wikipedia.org,0,192.3.161.207,414.396,left_only
4,unit_1,REP_FALSE_doc2,FALSE,A1HU0I8CI303XM,2,3,5,7,3,1,...,1,1,3,6,ITS VERY INTERESTING,en.wikipedia.org,5,192.3.161.207,139.195,left_only


2.890909090909091


In [7]:
df = pd.read_csv('../../Task/BigTask/Not_Poli_S100.csv')

m = df.groupby(['doc_id']).count()
m = m['unit_id']
print(m.mean())

6.7924528301886795
